In [2]:
import numpy as np

# The data

The following lists denote who each person follows. For example, John follows mark, sally, peter and tim while peter only follows tim.

The goal is to see who is ranked as most important based on the amount of followers each person has.

In [3]:
john = ['mark', 'sally', 'peter', 'tim']
mark = ['john', 'sally']
peter = ['tim']
sally = ['mark', 'john', 'peter', 'tim']
tim = ['sally']

Create a sorted list of all the people.

In [4]:
total = set(john + mark + sally + peter + tim)
total = sorted(list(total))
print(total)

['john', 'mark', 'peter', 'sally', 'tim']


## Building the Google Matrix

We initialize the google matrix to be a zeros matrix of dims (#_people, #_people).

In [5]:
s_mat = np.zeros(shape = (len(total), len(total)))
print(s_mat)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


We then traverse each person's list of the people they follow, denoting the importance (proportion of outgoing nodes) that each person dedicates to the other person. For example, since John (column 1) follows the other 4 people, he gives 1/4 of his importance to each other person. Peter (column 3), however, only follows 1 person (tim), so he transfers all of his importance to tim.

In [6]:
for i, per in enumerate([john, mark, peter, sally, tim]):
    for pub in per:
        if pub in total:
            s_mat[total.index(pub),i] = 1
        s_mat[total.index(pub),i] = s_mat[total.index(pub),i]/len(per)
        
print(s_mat)

[[0.   0.5  0.   0.25 0.  ]
 [0.25 0.   0.   0.25 0.  ]
 [0.25 0.   0.   0.25 0.  ]
 [0.25 0.5  0.   0.   1.  ]
 [0.25 0.   1.   0.25 0.  ]]


The PageRank theory holds that an imaginary surfer who is randomly clicking on links will eventually stop clicking. The probability, at any step, that the person will continue is a damping factor d. Various studies have tested different damping factors, but it is generally assumed that the damping factor will be set around 0.85.

The Google Matrix is constructed using the following equation:

$$G = \alpha*S + (1-\alpha) * {1\over{N}}$$

where $\alpha$ denotes the damping factor and $N$ denotes the number of documents.

In [7]:
damping_factor = 0.85

G_mat = s_mat * damping_factor

np.place(G_mat, G_mat == 0, (1-damping_factor) * 1/(len(total)))

In [8]:
print(G_mat)

[[0.03   0.425  0.03   0.2125 0.03  ]
 [0.2125 0.03   0.03   0.2125 0.03  ]
 [0.2125 0.03   0.03   0.2125 0.03  ]
 [0.2125 0.425  0.03   0.03   0.85  ]
 [0.2125 0.03   0.85   0.2125 0.03  ]]


To converge of the eigenvector of the Google matrix, which denotes the pagerank, we use the Power Method (Power Iteration) to iterate our way to a valid eigenvector. Initially, we choose a random, or partially approximated eigenvector, then we use the following recursive relation to approximate a valid eigenvector:

$$b_{k+1} = {{A*b_{k}}\over{||A*b_{k}||}}$$

In [9]:
def power_iteration(A, num_iters):
    rank = np.full((1, len(total)), 1/len(total))
    rank = rank.T
    
    for i in range(num_iters):
        rank_new = np.dot(A, rank)
        rank_norm = np.linalg.norm(rank_new)
        rank = rank_new/rank_norm
        
    return rank

## Results

The results are rather accurate, although they contain some minor discrepencies. Firstly, the ranks for john, mark, and peter are all around the ~0.275 range, which makes sense considering that all 3 of them are followed by only 2 other people.

Additionally, the ranks for sally and tim are greater than john, mark, and peter because they're both followed by 3 other people.

However, note that, even though sally and tim are followed by the same amount of people, the rank for sally is greater than the rank for tim. This is due to the fact that Sally has more *dedicated followers*. While Sally's followers, John, Mark, and Tim, follow 3, 1, and 0 people other than Sally, respectively, Tim's followers, John, Peter, and Sally, follow 3, 0, and 3 people other than Tim. Therefore, Sally's followers are more unique, that is, they are very interested in what Sally has to say, while Tim's followers are interested in what Tim has to say amongst what others have to say.

This is where the power lies in the concept of a Google matrix: the transfer of importance. While following a Markov Chain type model, the rank is capable of storing the importance that one person attributes to the other through who/how many people they follow.

In [10]:
rank = power_iteration(s_mat, 10)
print(rank)

[[0.30695658]
 [0.25589741]
 [0.25589741]
 [0.71640898]
 [0.51143803]]


## Extra: Getting the eigenvector

To obtain the eigenvalue for the eigenvector (pageRank) approximated above, we use the Raleigh Quotient, defined as follows:

$$R(M,x) = {{x*Mx}\over{x*x}}$$

Where M represents the matrix the eigenvector was derived from, in this case the Google Matrix, and x represents the eigenvector.

In [11]:
def raleighQuotient(m, rank):
    r = np.dot(rank.T, np.matmul(m, rank))
    r = np.squeeze(r)
    r = r/np.dot(rank.T, rank)
    return np.squeeze(r)

In [12]:
raleighQuotient(G_mat, rank)

array(0.9082324)

In [13]:
G_mat

array([[0.03  , 0.425 , 0.03  , 0.2125, 0.03  ],
       [0.2125, 0.03  , 0.03  , 0.2125, 0.03  ],
       [0.2125, 0.03  , 0.03  , 0.2125, 0.03  ],
       [0.2125, 0.425 , 0.03  , 0.03  , 0.85  ],
       [0.2125, 0.03  , 0.85  , 0.2125, 0.03  ]])

## Dynamic Google Matrix

In [239]:
user_buff = {}
total = {}

# User format: [str::name, int::uid, ]
john = ['john',0, ['mark', 'sally', 'peter', 'tim']]
mark = ['mark',1, ['john', 'sally']]
peter = ['peter',2, ['tim']]
sally = ['sally',3, ['mark', 'john', 'peter', 'tim']]
tim = ['tim',4, ['sally']]

In [240]:
# Updates the dictionary of user/followers. The key is the user id and the values are the people that the user follows. 
def add_users_to_buff(user_buff, usrs = []):
    for i in usrs:
        user_buff[i[1]] = i[2][:]
        
    return user_buff

# update the dictionary of user/id. The dict is unique, where the key is the name and the value is the user's id
def add_users_to_list(total, usrs = []):
    for i in usrs:
        if i[0] not in list(total.keys()):
            total[i[0]] = i[1]
        
    return total

# initialize a G_matrix with a list of users. 
def init_G_matrix(user_buff, total, damping_factor): 
    s_mat = np.zeros(shape = (len(total), len(total)))
    
    for i, per in enumerate(list(user_buff.keys())):
        for pub in user_buff[per]:
            if pub in total:
                s_mat[total[pub],i] = 1
            s_mat[total[pub],i] = s_mat[total[pub],i]/len(user_buff[per])
        if np.count_nonzero(s_mat[:,i]) == 0:
            s_mat[:,i].fill(1/len(total))
            
            
    G_mat = s_mat * damping_factor

#     np.place(G_mat, G_mat == 0, (1-damping_factor) * 1/(len(total)))
    
    return G_mat

# power method used to find the eigenvector of the google matrix
def power_iteration(A, num_users,num_iters):
    rank = np.full((1, num_users), 1/num_users)
    rank = rank.T
    
    for i in range(num_iters):
        rank_new = np.dot(A, rank)
        rank_norm = np.linalg.norm(rank_new)
        rank = rank_new/rank_norm
        
    return rank

# use the power method to compute the google rank
def compute_rank(g_mat, total, num_iters):
    
    return power_iteration(g_mat, len(total), num_iters)

In [241]:
user_buff = add_users_to_buff(user_buff, [john, mark, peter, sally, tim])
user_buff

{0: ['mark', 'sally', 'peter', 'tim'],
 1: ['john', 'sally'],
 2: ['tim'],
 3: ['mark', 'john', 'peter', 'tim'],
 4: ['sally']}

In [242]:
total = add_users_to_list(total, [john, mark, peter, sally, tim])
total

{'john': 0, 'mark': 1, 'peter': 2, 'sally': 3, 'tim': 4}

In [243]:
g_mat = init_G_matrix(user_buff, total, 0.85)
g_mat

array([[0.    , 0.425 , 0.    , 0.2125, 0.    ],
       [0.2125, 0.    , 0.    , 0.2125, 0.    ],
       [0.2125, 0.    , 0.    , 0.2125, 0.    ],
       [0.2125, 0.425 , 0.    , 0.    , 0.85  ],
       [0.2125, 0.    , 0.85  , 0.2125, 0.    ]])

In [244]:
compute_rank(g_mat, total, 100)

array([[0.30698671],
       [0.25582226],
       [0.25582226],
       [0.71630231],
       [0.51164451]])

In [245]:
mike = ['mike', 5, ['peter', 'sally']]

In [246]:
user_buff = add_users_to_buff(user_buff, [mike])
user_buff

{0: ['mark', 'sally', 'peter', 'tim'],
 1: ['john', 'sally'],
 2: ['tim'],
 3: ['mark', 'john', 'peter', 'tim'],
 4: ['sally'],
 5: ['peter', 'sally']}

In [247]:
total = add_users_to_list(total, [mike])
total

{'john': 0, 'mark': 1, 'mike': 5, 'peter': 2, 'sally': 3, 'tim': 4}

In [248]:
g_mat = init_G_matrix(user_buff, total, 0.85)
g_mat

array([[0.    , 0.425 , 0.    , 0.2125, 0.    , 0.    ],
       [0.2125, 0.    , 0.    , 0.2125, 0.    , 0.    ],
       [0.2125, 0.    , 0.    , 0.2125, 0.    , 0.425 ],
       [0.2125, 0.425 , 0.    , 0.    , 0.85  , 0.425 ],
       [0.2125, 0.    , 0.85  , 0.2125, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    ]])

In [238]:
compute_rank(g_mat, total, 100)

array([[0.29961076],
       [0.23968861],
       [0.26964968],
       [0.71906582],
       [0.50933829],
       [0.05992215]])